In [1]:
%%bash
pip install pyspark

In [2]:
%%bash
ls
# !ls

airlines.html
airlines.ipynb
airlines.pdf
data


In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F 

In [4]:
sc = SparkContext()
ss = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/01/10 20:42:47 WARN Utils: Your hostname, elitebook resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/01/10 20:42:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/10 20:42:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
txnRDD = sc.textFile("./data/airlines.csv")
txnRDD.take(5)

['Year,Quarter,Average revenue per seat,total no. of booked seats',
 '1995,1,296.9,46561',
 '1995,2,296.8,37443',
 '1995,3,287.51,34128',
 '1995,4,287.78,30388']

In [6]:
# ss.createDataFrame(data=txnRDD, )
df: DataFrame = ss.read.csv("./data/airlines.csv", header='True')
# txnDF = txnRDD.toDF(schema=txnRDD.take(1))
df.show(n=5)

+----+-------+------------------------+-------------------------+
|Year|Quarter|Average revenue per seat|total no. of booked seats|
+----+-------+------------------------+-------------------------+
|1995|      1|                   296.9|                    46561|
|1995|      2|                   296.8|                    37443|
|1995|      3|                  287.51|                    34128|
|1995|      4|                  287.78|                    30388|
|1996|      1|                  283.97|                    47808|
+----+-------+------------------------+-------------------------+
only showing top 5 rows



In [7]:
df = df.selectExpr("Year as year", "Quarter as qtr", "`Average revenue per seat` as avg_revenue", "`total no. of booked seats` as booked_seats")
df.show(n=5)

+----+---+-----------+------------+
|year|qtr|avg_revenue|booked_seats|
+----+---+-----------+------------+
|1995|  1|      296.9|       46561|
|1995|  2|      296.8|       37443|
|1995|  3|     287.51|       34128|
|1995|  4|     287.78|       30388|
|1996|  1|     283.97|       47808|
+----+---+-----------+------------+
only showing top 5 rows



In [8]:
df.describe()

DataFrame[summary: string, year: string, qtr: string, avg_revenue: string, booked_seats: string]

### Q1 Identifying the year with the highest number of people travelled

In [9]:
df1 = df.selectExpr("year", "booked_seats")
df1.describe()

DataFrame[summary: string, year: string, booked_seats: string]

In [10]:
df1.show(n=5)


+----+------------+
|year|booked_seats|
+----+------------+
|1995|       46561|
|1995|       37443|
|1995|       34128|
|1995|       30388|
|1996|       47808|
+----+------------+
only showing top 5 rows



In [11]:
df1.groupBy(F.col("year")).agg(F.sum(df1.booked_seats).alias("booked_seats")).sort("booked_seats", ascending=False).show(n=1)


+----+------------+
|year|booked_seats|
+----+------------+
|2007|    176299.0|
+----+------------+
only showing top 1 row



### Q2 Identifying the year with highest revenue

In [12]:
# df2 = df.selectExpr("Year", "Quarter", "`Average revenue per seat` as revenue")
df2 = df.selectExpr("year", "qtr", "avg_revenue", "booked_seats")
df2.describe()

DataFrame[summary: string, year: string, qtr: string, avg_revenue: string, booked_seats: string]

In [13]:
# total revenue per quarter
df2 = df2.withColumn("rev_per_qtr", df2.avg_revenue*df2.booked_seats)
df2.show(n=5)

+----+---+-----------+------------+--------------------+
|year|qtr|avg_revenue|booked_seats|         rev_per_qtr|
+----+---+-----------+------------+--------------------+
|1995|  1|      296.9|       46561|1.3823960899999999E7|
|1995|  2|      296.8|       37443|        1.11130824E7|
|1995|  3|     287.51|       34128|          9812141.28|
|1995|  4|     287.78|       30388|   8745058.639999999|
|1996|  1|     283.97|       47808|1.3576037760000002E7|
+----+---+-----------+------------+--------------------+
only showing top 5 rows



In [14]:
df2 = df2.groupBy(df2.year).agg(F.sum(df2.rev_per_qtr).alias("revenue"))

In [15]:
df2.sort("revenue", ascending=False).show(n=1)

+----+-------------+
|year|      revenue|
+----+-------------+
|2013|6.636320871E7|
+----+-------------+
only showing top 1 row



### Q3 Identifying the year and quarter highest revenue


In [16]:
df3 = df.selectExpr("year", "qtr", "avg_revenue", "avg_revenue * booked_seats as rev_per_qtr")
df3.describe()

DataFrame[summary: string, year: string, qtr: string, avg_revenue: string, rev_per_qtr: string]

In [17]:
df3.show(n=5)

+----+---+-----------+--------------------+
|year|qtr|avg_revenue|         rev_per_qtr|
+----+---+-----------+--------------------+
|1995|  1|      296.9|1.3823960899999999E7|
|1995|  2|      296.8|        1.11130824E7|
|1995|  3|     287.51|          9812141.28|
|1995|  4|     287.78|   8745058.639999999|
|1996|  1|     283.97|1.3576037760000002E7|
+----+---+-----------+--------------------+
only showing top 5 rows



In [18]:
df3.sort("rev_per_qtr", ascending=False).show(n=1)

+----+---+-----------+-------------+
|year|qtr|avg_revenue|  rev_per_qtr|
+----+---+-----------+-------------+
|2014|  4|     392.66|1.881940848E7|
+----+---+-----------+-------------+
only showing top 1 row

